# Present MAL data

In [1]:
import gzip
import glob

import numpy as np
import pandas as pd
import xmltodict as xtd
import qgrid
from rich import console, markdown

In [2]:
out = console.Console(width=60)
Md = markdown.Markdown

## Prepare Utilities

Loading function, from an XML file to:
- personal info, `pd.Series`
- anime/manga table, `pd.DataFrame`

In [3]:
def read_list(my_list, kind="anime"):
    dict_ = xtd.parse(my_list)["myanimelist"]
    myinfo_d = dict_["myinfo"]
    list_d = dict_[kind]

    myinfo = pd.Series(myinfo_d)
    list_df = pd.DataFrame(list_d)

    return myinfo, list_df

Apply types missing in XML

In [4]:
def to_numeric(df):
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
            out.print(f"[u]{col}[/u]:", Md(f"converted to **{df[col].dtype}**"))
        except ValueError:
            pass
    
    return df

def to_datetime(df):
    for col in df.columns:
        if "_date" in col:
            df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors="coerce")
            out.print(f"[u]{col}[/u]:", Md(f"converted to **{df[col].dtype}**"))
    
    return df

## Anime

In [5]:
anime_list_file = glob.glob("data/animelist*.gz")[0]

with gzip.open(anime_list_file, "r") as archive:
    my_anime_list = archive.read()

anime_info, anime = read_list(my_anime_list, "anime")

anime_info

user_id                    6962147
user_name                 Annibale
user_export_type                 1
user_total_anime               548
user_total_watching              7
user_total_completed           198
user_total_onhold               12
user_total_dropped               8
user_total_plantowatch         323
dtype: object

In [6]:
anime = to_numeric(anime)
anime = to_datetime(anime)

In [7]:
interesting_fields = [
    "series_title",
    "series_type",
    "series_episodes",
    "my_watched_episodes",
    "my_start_date",
    "my_finish_date",
    "my_score",
    "my_status",
    "my_tags",
]

anime_qw = qgrid.show_grid(anime[interesting_fields], show_toolbar=True)
anime_qw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
sublists = {}
for status in np.unique(anime["my_status"].values):
    sublists[status] = anime[anime["my_status"] == status]

In [9]:
sublists["Completed"][sublists["Completed"]["my_score"] == 0]

,series_animedb_id,series_title,series_type,series_episodes,my_id,my_watched_episodes,my_start_date,my_finish_date,my_rated,my_score,my_dvd,my_storage,my_status,my_comments,my_times_watched,my_rewatch_value,my_tags,my_rewatching,my_rewatching_ep,update_on_import
46,36456,Boku no Hero Academia 3rd Season,TV,25,0,25,NaT,2018-10-04,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
101,780,Detective Conan Movie 02: The Fourteenth Target,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
102,781,Detective Conan Movie 03: The Last Wizard of t...,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
103,1363,Detective Conan Movie 04: Captured in Her Eyes,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
106,1366,Detective Conan Movie 07: Crossroad in the Anc...,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
108,1505,Detective Conan Movie 09: Strategy Above the D...,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
109,1506,Detective Conan Movie 10: Requiem of the Detec...,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
110,2171,Detective Conan Movie 11: Jolly Roger in the D...,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
111,4447,Detective Conan Movie 12: Full Score of Fear,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0
112,5460,Detective Conan Movie 13: The Raven Chaser,Movie,1,0,1,NaT,NaT,NaN,0,NaN,NaN,Completed,None,0,None,None,0,0,0


## Manga

In [10]:
manga_list_file = glob.glob("data/mangalist*.gz")[0]

with gzip.open(manga_list_file, "r") as archive:
    my_manga_list = archive.read()

manga_info, manga = read_list(my_manga_list, "manga")

manga_info

user_id                   6962147
user_name                Annibale
user_export_type                2
user_total_manga               38
user_total_reading              9
user_total_completed            1
user_total_onhold               0
user_total_dropped              0
user_total_plantoread          28
dtype: object

In [11]:
manga = to_numeric(manga)
manga = to_datetime(manga)

In [12]:
manga_interesting_fields = [
    "manga_title",
    "manga_volumes",
    "manga_chapters",
    "my_read_volumes",
    "my_read_chapters",
    "my_start_date",
    "my_finish_date",
    "my_scanalation_group",
    "my_score",
    "my_status",
    "my_tags",
]

manga_qw = qgrid.show_grid(manga[manga_interesting_fields], show_toolbar=True)
manga_qw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…